In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
from src.leaflet_creation import create_map_visualization, get_color_for_df

In [4]:
true_pred = pd.read_csv('poster used true and pred.csv',index_col=0)

In [4]:
seg = gpd.read_file('data/segments_dc.geojson')

In [6]:
%matplotlib inline

In [7]:
simplified = seg[['geometry']].merge(true_pred, left_index=True, right_index=True, how='right')
simplified['diff'] = simplified.apply(lambda x: abs(x.csl-x.pred), axis=1)
true = simplified[['geometry', 'csl']].reset_index()
pred = simplified[['geometry', 'pred']].reset_index()
diff = simplified[['geometry', 'diff']].reset_index()
diff['diff'].value_counts()

In [6]:
def getColor(d):
    if d > 4: return '#1a9641'  # green (4, ]
    if d > 3: return '#a6d96a'  # light green (3, 4]
    if d > 2: return '#ffffbf'  # yellow: (2, 3]
    if d > 1: return '#fdae61'  # orange: (1, 2]
    return '#d7191c'            # red: (, 1]


In [24]:
true['color'] = true.csl.apply(getColor)
pred['color'] = pred.pred.apply(getColor)
diff['color'] = diff['diff'].apply(lambda x: getColor(5-x))

In [25]:
html_title = 'poster map'
file_path = ''
file_name = 'poster map'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets', 'stsg']
map_layers = ['light','streets', 'satellite']
binding_data=[['true_map','actual map'],['pred_map','predicted map'],['diff_map', 'true-pred diff']]
gpdfs = [true, pred, diff]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs, width=1000)

In [18]:
scheme_name='default'
scheme_name='experience'

pred_whole_dc = pd.read_csv('data/Pred_y_whole_dc_5lvl_spatial_%s.csv' % scheme_name,index_col=0)

whole = seg[['geometry']].merge(pred_whole_dc, left_index=True, right_index=True, how='right').reset_index()

whole['color'] = whole['whole-pred'].apply(getColor)

In [19]:
html_title = 'DC pred map ' + scheme_name
file_path = ''
file_name = 'DC pred map ' + scheme_name
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['osm_mapnik', 'stsg']
map_layers = ['dark_black','osm_mapnik']
binding_data=[['pred_map','predicted map']]
gpdfs = [whole]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs, width=2000, height=1500)